In [1]:
import pennylane as qml
import torch
from torch import nn
from torch.utils.data import DataLoader
from pennylane import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using {device} device')

Using cpu device


/home/franz/anaconda3/envs/fourier/lib/python3.8/site-packages/torch/cuda/__init__.py:80: UserWarning: CUDA initialization: CUDA unknown error - this may be due to an incorrectly set up environment, e.g. changing env variable CUDA_VISIBLE_DEVICES after program start. Setting the available devices to be zero. (Triggered internally at  /opt/conda/conda-bld/pytorch_1640811757556/work/c10/cuda/CUDAFunctions.cpp:112.)
  return torch._C._cuda_getDeviceCount() > 0


In [2]:
import sys
sys.path.insert(0, "../")
import utils.utils as utils
import models.fourier_models as fm
import models.quantum_models as qm

In [3]:
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split

In [4]:
# generate a suitable regression task
n_samples = 200
n_features = 3
n_informative = 3
n_targets = 1
noise = 0.0
random_state = 42
X, y = make_regression(n_samples=n_samples, n_features=n_features, n_informative=n_informative, n_targets=n_targets, noise=noise, random_state=random_state)
X, y = torch.from_numpy(X), torch.from_numpy(y)
# Scale data to interval [-pi/2, pi/2]
X_scaled = utils.data_scaler(X, interval=(-torch.pi/2, torch.pi/2))

# Split the data set into training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X_scaled, y, test_size=0.2, random_state=2)

# torch dataloaders
train_data_list = []
for i in range(len(X_train)):
    data_point = (X_train[i], y_train[i])
    train_data_list.append(data_point)

test_data_list = []
for i in range(len(X_test)):
    data_point = (X_test[i], y_test[i])
    test_data_list.append(data_point)
    
train_dataloader = DataLoader(train_data_list, batch_size=200, shuffle=True)
test_dataloader = DataLoader(test_data_list, batch_size=200, shuffle=True)

In [5]:
n_qubits = n_features
model = qm.QuantumRegressionModel(n_qubits)

In [6]:
def train(dataloader, model, loss_fn, optimizer, printing=False):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred.flatten(), y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            if printing == True:
                print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")
                for param_group in optimizer.param_groups:
                    print("lr: ", param_group['lr'])
        return loss

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred.flatten(), y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")
    return test_loss

In [7]:
loss_fn = nn.MSELoss(reduction='mean') # equiv. to torch.linalg.norm(input-target)**2
optimizer = torch.optim.Adam(model.parameters(), lr=0.8)
train(train_dataloader, model, loss_fn, optimizer, printing=True)
optimizer = torch.optim.Adam(model.parameters(), lr=0.2)

epochs = 100
for t in tqdm(range(epochs)):
    # print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, printing=True)
    # test(test_dataloader, model, loss_fn)
    # scheduler.step()
print("Done!")
print(train(train_dataloader, model, loss_fn, optimizer, printing=True))
print(test(test_dataloader, model, loss_fn))

loss: 10550.017158  [    0/  160]
lr:  0.8


  1%|          | 1/100 [00:03<06:25,  3.90s/it]

loss: 10512.832161  [    0/  160]
lr:  0.2


  2%|▏         | 2/100 [00:07<06:21,  3.89s/it]

loss: 10430.224336  [    0/  160]
lr:  0.2


...

 99%|█████████▉| 99/100 [06:26<00:03,  3.88s/it]

loss: 5327.220246  [    0/  160]
lr:  0.2


100%|██████████| 100/100 [06:30<00:00,  3.90s/it]

loss: 5288.198656  [    0/  160]
lr:  0.2
Done!


loss: 5246.066722  [    0/  160]
lr:  0.2
5246.066721595605
Test Error: 
 Accuracy: 0.0%, Avg loss: 3764.587826 

3764.587826416885


In [8]:
# generate frequencies
max_freq = 2
dim = X_scaled[0].shape[0]

W = utils.freq_generator(max_freq, dim)

# compute best approximation
ba_coeffs = utils.fourier_best_approx(W, X_train, y_train)

print("training_loss: ",utils.loss(W, ba_coeffs, X_train, y_train))
print("test loss: ",utils.loss(W, ba_coeffs, X_test, y_test))

training_loss:  tensor(0.0310, dtype=torch.float64)
test loss:  tensor(0.5630, dtype=torch.float64)


In [14]:
model(X_train[0])

tensor([-52.9373], dtype=torch.float64, grad_fn=<SqueezeBackward3>)

In [13]:
y_train[0]

tensor(-76.4457, dtype=torch.float64)

In [11]:
utils.loss(W, ba_coeffs, X_train[0:2], y_train[0:2])

tensor(0.3404, dtype=torch.float64)

In [12]:
epochs = 100
for t in tqdm(range(epochs)):
    # print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, printing=True)
    # test(test_dataloader, model, loss_fn)
    # scheduler.step()
print("Done!")
print(train(train_dataloader, model, loss_fn, optimizer, printing=True))
print(test(test_dataloader, model, loss_fn))

  1%|          | 1/100 [00:03<06:24,  3.89s/it]

loss: 5206.724404  [    0/  160]
lr:  0.2


  2%|▏         | 2/100 [00:07<06:30,  3.99s/it]

loss: 5175.474531  [    0/  160]
lr:  0.2


  3%|▎         | 3/100 [00:11<06:19,  3.91s/it]

...

 99%|█████████▉| 99/100 [06:25<00:03,  3.90s/it]

loss: 2670.430919  [    0/  160]
lr:  0.2


100%|██████████| 100/100 [06:29<00:00,  3.90s/it]

loss: 2608.851343  [    0/  160]
lr:  0.2
Done!


loss: 2612.068807  [    0/  160]
lr:  0.2
2612.0688069459147
Test Error: 
 Accuracy: 0.0%, Avg loss: 1601.819688 

1601.8196875253097


In [15]:
W = utils.freq_generator(max_freq, dim).to(device)
model = fm.Fourier_model(W)
model.to(device)
loss_fn = nn.MSELoss(reduction='mean') # equiv. to torch.linalg.norm(input-target)**2
optimizer = torch.optim.Adam(model.parameters(), lr=0.2)

epochs = 200
for t in tqdm(range(epochs)):
    # print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer, printing=True)
    # test(test_dataloader, model, loss_fn)
    # scheduler.step()
print("Done!")
print(train(train_dataloader, model, loss_fn, optimizer, printing=True))
print(test(test_dataloader, model, loss_fn))

 70%|███████   | 140/200 [00:00<00:00, 707.15it/s]

loss: 10536.059660  [    0/  160]
lr:  0.2
loss: 10050.367515  [    0/  160]
lr:  0.2
loss: 9588.613246  [    0/  160]
lr:  0.2
loss: 9150.663023  [    0/  160]
lr:  0.2
loss: 8735.713804  [    0/  160]
lr:  0.2
loss: 8343.156592  [    0/  160]
lr:  0.2
loss: 7972.735636  [    0/  160]
lr:  0.2
loss: 7624.208262  [    0/  160]
lr:  0.2
loss: 7297.212123  [    0/  160]
lr:  0.2
loss: 6991.237904  [    0/  160]
lr:  0.2
loss: 6705.645401  [    0/  160]
lr:  0.2
loss: 6439.703227  [    0/  160]
lr:  0.2
loss: 6192.630106  [    0/  160]
lr:  0.2
loss: 5963.618918  [    0/  160]
lr:  0.2
loss: 5751.840501  [    0/  160]
lr:  0.2
loss: 5556.436570  [    0/  160]
lr:  0.2
loss: 5376.512036  [    0/  160]
lr:  0.2
loss: 5211.132126  [    0/  160]
lr:  0.2
loss: 5059.325473  [    0/  160]
lr:  0.2
loss: 4920.092317  [    0/  160]
lr:  0.2
loss: 4792.416413  [    0/  160]
lr:  0.2
loss: 4675.279190  [    0/  160]
lr:  0.2
loss: 4567.674936  [    0/  160]
lr:  0.2
loss: 4468.625886  [    0/  160]

100%|██████████| 200/200 [00:00<00:00, 706.11it/s]

loss: 2201.906521  [    0/  160]
lr:  0.2
loss: 2194.728726  [    0/  160]
lr:  0.2
loss: 2187.591161  [    0/  160]
lr:  0.2
loss: 2180.493815  [    0/  160]
lr:  0.2
loss: 2173.436672  [    0/  160]
lr:  0.2
loss: 2166.419704  [    0/  160]
lr:  0.2
loss: 2159.442877  [    0/  160]
lr:  0.2
loss: 2152.506146  [    0/  160]
lr:  0.2
loss: 2145.609459  [    0/  160]
lr:  0.2
loss: 2138.752753  [    0/  160]
lr:  0.2
loss: 2131.935961  [    0/  160]
lr:  0.2
loss: 2125.159006  [    0/  160]
lr:  0.2
loss: 2118.421806  [    0/  160]
lr:  0.2
loss: 2111.724271  [    0/  160]
lr:  0.2
loss: 2105.066309  [    0/  160]
lr:  0.2
loss: 2098.447820  [    0/  160]
lr:  0.2
loss: 2091.868702  [    0/  160]
lr:  0.2
loss: 2085.328847  [    0/  160]
lr:  0.2
loss: 2078.828144  [    0/  160]
lr:  0.2
loss: 2072.366479  [    0/  160]
lr:  0.2
loss: 2065.943733  [    0/  160]
lr:  0.2
loss: 2059.559785  [    0/  160]
lr:  0.2
loss: 2053.214510  [    0/  160]
lr:  0.2
loss: 2046.907778  [    0/  160]
l